# Настройка окружения и загрузка данных
 Здесь подключаем pandas для работы с таблицами, numpy для численных операций
 и pathlib для формирования пути к файлу.
 Устанавливаем имена столбцов и считываем CSV с письмами в DataFrame.
 Затем выводим первые 10 строк для проверки корректности загрузки.


In [6]:
import numpy as np
import pandas as pd
from pathlib import Path

dir_path = Path.cwd()
name_dataset = "emails.csv"
emails = pd.read_csv('emails.csv')
print("Первые 10 писем:")
print(emails.head(10))


Первые 10 писем:
                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1
5  Subject: great nnews  hello , welcome to medzo...     1
6  Subject: here ' s a hot play in motion  homela...     1
7  Subject: save your money buy getting this thin...     1
8  Subject: undeliverable : home based business f...     1
9  Subject: save your money buy getting this thin...     1


# Предобработка текста в список уникальных слов

Определяем функции для обработки текста:
 1) split_string_into_unique_words: разбивает строку на слова и оставляет уникальные.
 2) process_series_email       : применяет это преобразование ко всему столбцу text.
 Сохраняем результат в новом столбце 'words' и выводим первые 10 строк.


In [7]:
def split_string_into_unique_words(string):
    return list(set(string.lower().split()))

def process_series_email(series_text):
    """Преобразует Series текстов в Series списков уникальных слов"""
    return series_text.apply(split_string_into_unique_words)

column_emails = 'text'
column_words = 'words'
emails[column_words] = process_series_email(emails[column_emails])
print("После предобработки (столбец 'words'):")
print(emails.head(10))

После предобработки (столбец 'words'):
                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   
5  Subject: great nnews  hello , welcome to medzo...     1   
6  Subject: here ' s a hot play in motion  homela...     1   
7  Subject: save your money buy getting this thin...     1   
8  Subject: undeliverable : home based business f...     1   
9  Subject: save your money buy getting this thin...     1   

                                               words  
0  [website, for, ordered, with, through, drafts,...  
1  [waterway, fanny, chameleon, palfrey, bedtime,...  
2  [unbelievable, loan, im, been, for, wanting, a...  
3  [goldengraphix, for, format, version, graphix,...  
4  

# Вычисление априорных вероятностей классов

 Считаем общее число писем и число спамов. Вычисляем P(spam) и P(ham).

In [8]:
column_label = 'spam'
num_emails = len(emails)
counts_label = emails[column_label].value_counts()
num_spam = counts_label.get(1, 0)
num_ham = counts_label.get(0, 0)
print("Распределение меток (ham=0, spam=1):")
print(counts_label)
print(f"P(spam) = {num_spam/num_emails:.4f}\n")

Распределение меток (ham=0, spam=1):
spam
0    4360
1    1368
Name: count, dtype: int64
P(spam) = 0.2388



# БЛОК 4: Построение словарной модели наивного байеса

 Для каждого слова считаем, в скольких spam и ham письмах оно встречается.
 Используем Лапласово сглаживание (начальные счётчики = 1) для избежания нулей.


In [9]:
model = {}  # {word: {'spam': count, 'ham': count}}
for _, row in emails.iterrows():
    is_spam = row[column_label] == 1
    for word in row[column_words]:
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1}
        model[word]['spam' if is_spam else 'ham'] += 1

# Просмотр примеров
print("Пример model['lottery']     =", model.get('lottery'))
print("Пример model['sale']        =", model.get('sale'), "\n")

Пример model['lottery']     = {'spam': 9, 'ham': 1}
Пример model['sale']        = {'spam': 39, 'ham': 42} 



# Функция predict_bayes для одного слова

Возвращает P(spam|word) = count_spam/(count_spam+count_ham).

In [10]:
def predict_bayes(word, model):
    counts = model.get(word.lower())
    if not counts:
        return None
    return counts['spam'] / (counts['spam'] + counts['ham'])

print("P(spam|'lottery') =", predict_bayes('lottery', model))
print("P(spam|'sale')    =", predict_bayes('sale', model), "\n")

P(spam|'lottery') = 0.9
P(spam|'sale')    = 0.48148148148148145 



# Полная функция predict_naive_bayes для целого письма

 Вычисляет P(spam|email) ∝ P(spam) * ∏ P(word|spam)
 и P(ham|email) аналогично, затем нормирует.


In [11]:
def predict_naive_bayes(email_text, model, num_spam, num_ham, num_emails):
    words = set(email_text.lower().split())
    p_spam = num_spam / num_emails
    p_ham = num_ham / num_emails
    for w in words:
        counts = model.get(w)
        if counts:
            p_spam *= counts['spam'] / num_spam
            p_ham *= counts['ham'] / num_ham
    total = p_spam + p_ham
    return p_spam / total if total > 0 else 0.5

# Тестируем на примерах
tests = [
    'lottery sale',
    'Hi mom how are you',
    'enter the lottery to win three million dollars',
    'meet me at the lobby of the hotel at nine am'
]
for t in tests:
    print(f"P(spam|'{t}') = {predict_naive_bayes(t, model, num_spam, num_ham, num_emails):.4f}")

P(spam|'lottery sale') = 0.9638
P(spam|'Hi mom how are you') = 0.1255
P(spam|'enter the lottery to win three million dollars') = 0.9995
P(spam|'meet me at the lobby of the hotel at nine am') = 0.0001
